In [1]:
!nvidia-smi

Wed May 19 15:10:55 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 465.19.01    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   38C    P0    30W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [1]:
from google.colab import drive
from google.colab import output
drive.mount('/content/drive')

KeyboardInterrupt: ignored

In [ ]:
!rm -rf ml

In [ ]:
!mkdir /content/ml

mkdir: cannot create directory ‘/content/ml’: File exists


In [ ]:
!cp /content/drive/MyDrive/ml/htc_swin_fpn_v13_step_kfold.zip /content/ml
!cp /content/drive/MyDrive/ml/pths.zip /content/ml
!cp /content/drive/MyDrive/ml/input.zip /content/ml

In [ ]:
%cd ml

/content/ml


In [ ]:
!unzip "htc_swin_fpn_v13_step_kfold.zip" -d "./"
!unzip "input.zip" -d "./input"
!unzip "pths.zip" -d "./"
output.clear()

In [ ]:
%cd /content/ml/mmdetection_trash

/content/ml/mmdetection_trash


In [ ]:
|!apt-get install libsm6 libxext6 libxrender-dev
!pip install torch==1.6.0
!pip install torchvision==0.7.0
!pip install mmcv-full==latest+torch1.6.0+cu101 -f https://download.openmmlab.com/mmcv/dist/index.html
!pip install timm
!pip install terminaltables 
!pip install instaboostfast
!pip uninstall pycocotools --yes
!pip uninstall mmpycocotools --yes
!pip install mmpycocotools
!pip install wandb
output.clear()

In [ ]:
from mmcv import Config
from mmdet.datasets import build_dataset
from mmdet.models import build_detector
from mmdet.apis import train_detector
from mmdet.datasets import (build_dataloader, build_dataset,
                            replace_ImageToTensor)

apex is not installed
apex is not installed
apex is not installed
apex is not installed


In [ ]:
# config file 들고오기
cfg = Config.fromfile('./configs/swin/htc_swin_fpn_v13_4.py')
cfg.data.samples_per_gpu = 3
cfg.seed = 1104
cfg.gpu_ids = [0]
cfg.work_dir = '/content/drive/MyDrive/ml/work_dirs/htc_swin_fpn_v13_4'

In [ ]:
group_name = 'cascade'; project_name = 'kw_ojd_p3_2'; run_name = "htc_swin_fpn_v13_4"
config_list = {
    'epoch' : cfg.runner.max_epochs,
    'batch_size' :  cfg.data.samples_per_gpu,
    'optimizer' : cfg.optimizer,
    'optimizer_config' : cfg.optimizer_config,
    'lr_config' : cfg.lr_config
}
cfg.log_config.hooks[1].init_kwargs['group'] = group_name # group name(option)
cfg.log_config.hooks[1].init_kwargs['name'] = run_name # run name
cfg.log_config.hooks[1].init_kwargs['config'] = config_list # config

In [ ]:
model = build_detector(cfg.model)

2021-05-17 18:10:01,685 - mmdet - INFO - load model from: /content/ml/pretrained_pth/swin_base_patch4_window7_224.pth
2021-05-17 18:10:07,224 - mmdet - WARNING - The model and loaded state dict do not match exactly

unexpected key in source state_dict: norm.weight, norm.bias, head.weight, head.bias, layers.0.blocks.1.attn_mask, layers.1.blocks.1.attn_mask, layers.2.blocks.1.attn_mask, layers.2.blocks.3.attn_mask, layers.2.blocks.5.attn_mask, layers.2.blocks.7.attn_mask, layers.2.blocks.9.attn_mask, layers.2.blocks.11.attn_mask, layers.2.blocks.13.attn_mask, layers.2.blocks.15.attn_mask, layers.2.blocks.17.attn_mask

missing keys in source state_dict: norm0.weight, norm0.bias, norm1.weight, norm1.bias, norm2.weight, norm2.bias, norm3.weight, norm3.bias



In [ ]:
from mmcv.runner import load_checkpoint
checkpoint = load_checkpoint(model, "../pretrained_pth/cascade_mask_rcnn_swin_base_patch4_window7.pth", map_location='cuda:0')

Use load_from_local loader
The model and loaded state dict do not match exactly

size mismatch for roi_head.bbox_head.0.fc_cls.weight: copying a param with shape torch.Size([81, 1024]) from checkpoint, the shape in current model is torch.Size([12, 1024]).
size mismatch for roi_head.bbox_head.0.fc_cls.bias: copying a param with shape torch.Size([81]) from checkpoint, the shape in current model is torch.Size([12]).
size mismatch for roi_head.bbox_head.0.fc_reg.weight: copying a param with shape torch.Size([320, 1024]) from checkpoint, the shape in current model is torch.Size([44, 1024]).
size mismatch for roi_head.bbox_head.0.fc_reg.bias: copying a param with shape torch.Size([320]) from checkpoint, the shape in current model is torch.Size([44]).
size mismatch for roi_head.bbox_head.1.fc_cls.weight: copying a param with shape torch.Size([81, 1024]) from checkpoint, the shape in current model is torch.Size([12, 1024]).
size mismatch for roi_head.bbox_head.1.fc_cls.bias: copying a param wi

In [ ]:
datasets = [build_dataset(cfg.data.train)]

loading annotations into memory...
Done (t=10.22s)
creating index...
index created!


In [ ]:
train_detector(model, datasets[0], cfg, distributed=False, validate=True)

loading annotations into memory...


2021-05-17 18:10:32,812 - mmdet - INFO - Start running, host: root@bd48ca592b9f, work_dir: /content/drive/MyDrive/ml/work_dirs/casmaskrcnn_swin_base_p4_win7_adamw_Crunner_trash_13
2021-05-17 18:10:32,813 - mmdet - INFO - workflow: [('train', 1)], max: 24 epochs


Done (t=2.49s)
creating index...
index created!


wandb: Currently logged in as: pstage12 (use `wandb login --relogin` to force relogin)


2021-05-17 18:10:45,915 - mmdet - INFO - Epoch [1][10/873]	lr: 9.491e-07, eta: 4:40:16, time: 0.803, data_time: 0.237, memory: 9998, loss_rpn_cls: 0.5450, loss_rpn_bbox: 0.0618, s0.loss_cls: 2.7359, s0.acc: 2.0898, s0.loss_bbox: 0.3190, s0.loss_mask: 3.4106, s1.loss_cls: 1.1730, s1.acc: 2.8581, s1.loss_bbox: 0.0523, s1.loss_mask: 1.1908, s2.loss_cls: 0.5571, s2.acc: 12.0638, s2.loss_bbox: 0.0069, s2.loss_mask: 0.6790, loss: 10.7314
2021-05-17 18:10:51,261 - mmdet - INFO - Epoch [1][20/873]	lr: 1.948e-06, eta: 3:53:20, time: 0.535, data_time: 0.016, memory: 10038, loss_rpn_cls: 0.4242, loss_rpn_bbox: 0.0541, s0.loss_cls: 2.7164, s0.acc: 2.2461, s0.loss_bbox: 0.2869, s0.loss_mask: 4.1248, s1.loss_cls: 1.1593, s1.acc: 3.3073, s1.loss_bbox: 0.0475, s1.loss_mask: 0.8093, s2.loss_cls: 0.5512, s2.acc: 23.5937, s2.loss_bbox: 0.0067, s2.loss_mask: 0.6116, loss: 10.7921
2021-05-17 18:10:56,602 - mmdet - INFO - Epoch [1][30/873]	lr: 2.947e-06, eta: 3:37:33, time: 0.534, data_time: 0.013, memory: 

...

2021-05-17 22:22:43,681 - mmdet - INFO - Saving checkpoint at 24 epochs


DONE (t=0.54s).
Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.311
Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=1000 ] = 0.438
Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=1000 ] = 0.333
Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=1000 ] = 0.058
Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=1000 ] = 0.271
Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=1000 ] = 0.543
Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.441
Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=300 ] = 0.441
Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=1000 ] = 0.441
Average Recall     (AR) @[ IoU=0.50:0.95 | area= small | maxDets=1000 ] = 0.133
Average Recall     (AR) @[ IoU=0.50:0.95 | area=medium | maxDets=1000 ] = 0.415
Average Recall     (AR) @[ IoU=0.50:0.95 | area= large | maxDets=1000 ] = 0.729


2021-05-17 22:22:52,686 - mmdet - INFO - Epoch(val) [24][873]	bbox_mAP: 0.3330, bbox_mAP_50: 0.4410, bbox_mAP_75: 0.3520, bbox_mAP_s: 0.0800, bbox_mAP_m: 0.3010, bbox_mAP_l: 0.5740, bbox_mAP_copypaste: 0.333 0.441 0.352 0.080 0.301 0.574, segm_mAP: 0.3110, segm_mAP_50: 0.4380, segm_mAP_75: 0.3330, segm_mAP_s: 0.0580, segm_mAP_m: 0.2710, segm_mAP_l: 0.5430, segm_mAP_copypaste: 0.311 0.438 0.333 0.058 0.271 0.543


train/loss_rpn_cls,0.04868
train/loss_rpn_bbox,0.02777
train/s0.loss_cls,0.11303
train/s0.acc,96.33463
train/s0.loss_bbox,0.14734
train/s0.loss_mask,0.23175
train/s1.loss_cls,0.05533
train/s1.acc,96.16682
train/s1.loss_bbox,0.0662
train/s1.loss_mask,0.10228
train/s2.loss_cls,0.0259


train/loss_rpn_cls,█▆▅▆▆▃▅▅▃▃▂▅▄▄▃▃▃▂▃▃▂▂▃▁▂▂▁▁▃▂▂▃▂▂▃▃▂▂▂▂
train/loss_rpn_bbox,█▆▃▅▅▃▇▄▄▄▃▆▃▆▅▄▅▂▅▄▃▃▂▂▃▃▂▁▅▃▃▆▃▄▄▅▄▂▃▃
train/s0.loss_cls,█▇▅▅▄▃▄▄▃▃▃▅▃▄▃▃▃▂▃▄▂▂▂▂▂▁▁▁▂▂▂▄▂▂▂▃▂▁▃▂
train/s0.acc,▁▃▄▅▄▆▄▅▆▆▆▄▅▅▆▅▅▇▅▅▇▇▆▇▇▇██▆▆▆▅▇▆▆▆▆█▆▇
train/s0.loss_bbox,█▅▃▄▅▃▅▃▃▃▂▅▃▄▃▃▄▁▄▃▂▃▂▂▂▂▁▁▃▃▂▅▂▃▄▄▃▂▃▂
train/s0.loss_mask,█▅▃▃▃▃▃▃▃▂▂▃▂▂▂▂▂▁▂▂▁▁▁▁▁▁▁▁▂▂▂▂▁▂▂▂▁▂▂▂
train/s1.loss_cls,▇█▆▆▅▅▅▅▄▄▄▅▅▅▃▄▄▂▄▄▃▂▃▂▂▂▂▁▂▂▂▄▂▃▃▃▂▁▃▂
train/s1.acc,▃▁▃▃▃▅▃▄▅▄▅▄▄▄▅▅▅▇▄▄▆▇▆▇▇▇▇█▆▆▆▄▇▆▅▆▇█▅▇
train/s1.loss_bbox,▅█▄▆▇▅▇▄▄▄▄█▆▇▆▅▇▁▆▅▄▃▃▄▄▄▁▂▅▄▄█▄▆▅▅▅▂▄▄
train/s1.loss_mask,█▅▃▃▃▂▃▃▂▂▂▃▂▃▂▃▂▁▂▃▁▁▁▁▂▁▁▁▂▂▂▂▁▁▂▂▂▁▂▁
train/s2.loss_cls,▅█▆▆▆▅▅▅▄▄▄▅▅▅▃▄▄▂▄▄▃▂▃▂▂▂▂▁▂▃▂▄▃▃▃▃▂▁▃▂
